In [1]:
import time
import numpy as np
import string

#sklearn
from sklearn.metrics.pairwise import cosine_similarity

#bert
from sentence_transformers import SentenceTransformer

# FAQ Bot

In [2]:
FAQ_list = ['如何加入亚盈？', 
            '我可以直接在网络上存款吗？', 
            '我在哪里可以找到游戏的规则？', 
            '我可以只试玩而不下注吗？', 
            '你们的游戏会用多少副牌？', 
            '您们何时会洗牌？',
            '我的注码的限制是多少？',
            '如果忘记密码怎么办？', 
            '当你注册时出现，姓名已注册？',
            '一定要填附言么？', 
            '是否要手动额度转换？']
FAQ_Answer = ['您好， 您可以直接点选 注册，确实填写数据后，可立即登记成为亚盈会员。',
              '您好，可以，亚盈提供多种在线存款选择，详情请参照 存款帮助。',
              '您好，在未登入前，您可以在游戏的最外层看到"游戏规则"选项，清楚告诉您游戏的玩法、规则及派彩 方式。 在游戏窗口中，也有"规则"选项，让您在享受游戏乐趣的同时，可以弹跳窗口随时提醒您游戏规则。',\
              '您好，可以，我们诚挚邀请您观摩我们的游戏。只需注册一个会员账号，登陆后即可查看所有的游戏。',
              '您好，在百家乐我们会用8副牌，其他游戏则会根据其性质有所调整。',
              '您好，所有纸牌游戏，当红的洗牌记号出现或游戏因线路问题中断时便会进行重新洗牌。',
              '您好，从最低注单 10(视讯) 元人民币以上即可投注， 您的注码会根据您的存款有所不同，以及您挑选的游戏不同而有所区别。',
              '您好，您可以联系24小时在线客服人员咨询协助取回您的账号密码。',
              '您好，您可通过24小时在线客服人员协助处理。',
              '在您通过转账时，请一定要在备注/附言/等处填写附言编码，否则无法自动到账。',
              '亚盈开启全自动额度转换模式，只需直接进入游戏额度会自动转换，无需人工转换。']

In [3]:
import pandas as pd
temp = pd.DataFrame({'question':FAQ_list, 'answer':FAQ_Answer})

In [4]:
#temp.to_csv('C://Users//user//Desktop//faq.csv')

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('C://Users//admin//Desktop//faq.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
temp.to_excel(writer, sheet_name='Sheet1')

writer.save()

In [4]:
def remove_punctuation(s): 
    punctuation = '''''？表情\n!！()-[]{};:'"\,<>./[email protected]#$%^&*_~'''+ string.whitespace+ string.punctuation#'''''表情\n!！()-[]{};:'"\,<>./[email protected]#$%^&*_~'''
    #'''''\n!()-[]{};:'"\,<>./[email protected]#$%^&*_~！（）－［］｛｝；：’＂＼，＜＞．／＃＄％︿＆＊＿～'''
    my_str = s 
    no_punct = "" 
    for char in my_str: 
        if char not in punctuation: 
            no_punct += char 
    return(no_punct) 

def FAQ_BOT(customer_question, sentence_embeddings, answer, model, threshold, control):
    test_embeddings = model.encode(customer_question)
    similarity = cosine_similarity([test_embeddings], 
                                   sentence_embeddings)
    max_cos_similar = np.max(similarity)
    probsarg = [x for x in similarity.argsort()[0][-4:][::-1] ] 
    top4_similarty = [similarity[0][j] for j in probsarg]
    
    possibe_question = [FAQ_list[j] for j in probsarg]    
    most_likely_question = possibe_question[0]
    possibe_question = possibe_question[1:]
   
    if max_cos_similar >= threshold:
        max_idx = np.argmax(similarity)
        most_likely_ans = answer[max_idx]
        res = 1
    else:
        most_likely_ans = '这不在FAQ中'
        res = 0
    if control == 'jy_test':
        return most_likely_ans, res, most_likely_question, possibe_question, top4_similarty    
    elif control == 'prod':
        return most_likely_ans, res

In [5]:
FAQ_model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')
FAQ_list = [remove_punctuation(j) for j in FAQ_list]
faq_embeddings = FAQ_model.encode(FAQ_list)

In [ ]:
#s = time.time()
#FAQ_BOT('一定要填附言?', faq_embeddings, FAQ_Answer, FAQ_model, threshold=0.8, control='jy_test')
#print(time.time()-s)